
# Content Based Recommendation (İçerik Temelli Tavsiye)



## Film Overview'larına Göre Tavsiye Geliştirme


### 1. TF-IDF Matrisinin Oluşturulması
### 2. Cosine Similarity Matrisinin Oluşturulması
### 3. Benzerliklere Göre Önerilerin Yapılması
### 4. Çalışma Scriptinin Hazırlanması

In [1]:
#################################
# 1. TF-IDF Matrisinin Oluşturulması
#################################

In [3]:
#Gerekli kütüphaneleri ve methotları import ettik ve veriyi okuduk
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# https://www.kaggle.com/rounakbanik/the-movies-dataset
df = pd.read_csv("../datasets/movies_metadata.csv", low_memory=False)  # DtypeWarning kapamak icin

In [4]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
df.shape

(45466, 24)

In [6]:
#Overview kısmını vektörleştirmek için TfdifVectorizer objesi oluşturalım
tf_idf=TfidfVectorizer(stop_words="english") 
# stop_words parametresine metnin yazıldığı dil girilir
# ve o dilde çok kullanılan kelimelerin (the, on, at gibi) uçurulmasını sağlar

In [7]:
df["overview"].isnull().sum()

954

In [8]:
# eksik değerleri " " ile dolduralım
df["overview"]=df["overview"].fillna(" ")

In [26]:
tf_idf_matrix=tf_idf.fit_transform(df["overview"])
print("tf_idf_matrix'inin satırlarında {} tane film açıklamaları,".format(tf_idf_matrix.shape[0]),
      "sütunlarında ise bu filmlerin overview'inde geçen {} tane eşsiz kelime bulunur."
      .format(tf_idf_matrix.shape[1]))                    


tf_idf_matrix'inin satırlarında 45466 tane film açıklamaları, sütunlarında ise bu filmlerin overview'inde geçen 75827 tane eşsiz kelime bulunur.


In [19]:
#################################
# 2. Cosine Similarity Matrisinin Oluşturulması
#################################

In [28]:
#Cosine similarity fonksiyonuyla  bir filmin diğer tüm filmler ile olan benzerliklerini bulalım
#cosine_sim satır ve sütunları filmlerden oluşan bir square matrixdir
#Filmlerin kesişiminde o filmler arasındaki kosünüz benzerlikleri yazar
cosine_sim = cosine_similarity(tf_idf_matrix,
                               tf_idf_matrix)
cosine_sim.shape

(45466, 45466)

In [30]:
#################################
# 3. Benzerliklere Göre Önerilerin Yapılması
#################################

In [32]:
#Oluşturduğumuz matrixe öneri yapabilmemiz için filmlerin isminide eklememiz lazım
film_names=pd.Series(df.index,index=df["title"])
film_names

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64

In [33]:
film_names.index.value_counts()
#Görüldüğü gibi filmlerde çoklama var
#Burda farklı yaklaşımlar izlenebilir biz en güncel olanı tutucaz diğerlerini droplayacağız

Cinderella              11
Hamlet                   9
Alice in Wonderland      9
Beauty and the Beast     8
Les Misérables           8
                        ..
Golden Swallow           1
Shoot on Sight           1
Kill Bill: Vol. 2        1
Raiders of the Sun       1
Bestiary                 1
Name: title, Length: 42277, dtype: int64

In [34]:
film_names = film_names[~film_names.index.duplicated(keep='last')]
film_names.index.value_counts()

The Princess Comes Across        1
Kingdom of Crooked Mirrors       1
Empire of Silver                 1
Kamikaze Taxi                    1
A Barefoot Dream                 1
                                ..
A Buddha                         1
Blockade                         1
The Giant King                   1
Providence                       1
The Girl from the Marsh Croft    1
Name: title, Length: 42277, dtype: int64

In [37]:
#Şimdi seçtiğimiz bir filmler diğer bütün filmlerin benzerlik skorunu gözlemleyelim
movie_index=film_names["Sherlock Holmes"]
similarity_scores=pd.DataFrame(cosine_sim[movie_index],columns=["SCORE"])
similarity_scores

,SCORE
0,0.000000
1,0.003928
2,0.004768
3,0.000000
4,0.000000
...,...
45461,0.000000
45462,0.000000
45463,0.000000
45464,0.006792


In [38]:
#Sherlock Holmes filmine en çok benzeyen 10 filme bakalım
film_indices=similarity_scores.sort_values("SCORE",ascending=False)[1:11].index
df.title.iloc[film_indices]

34737    Приключения Шерлока Холмса и доктора Ватсона: ...
14821                                    The Royal Scandal
34750    The Adventures of Sherlock Holmes and Doctor W...
9743                           The Seven-Per-Cent Solution
4434                                        Without a Clue
29706                       How Sherlock Changed the World
18258                   Sherlock Holmes: A Game of Shadows
24665     The Sign of Four: Sherlock Holmes' Greatest Case
6432                   The Private Life of Sherlock Holmes
29154                          Sherlock Holmes in New York
Name: title, dtype: object

In [39]:
#################################
# 4. Çalışma Scriptinin Hazırlanması
#################################

In [42]:
def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['overview'] = dataframe['overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['overview'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [44]:
def content_based_recommender(title,dataframe):
    #cosine_sim matrixini oluşturalım
    cosnine_sim=calculate_cosine_sim(dataframe)
    # index'leri olusturma
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    # title'ın index'ini yakalama
    movie_index = indices[title]
    # title'a gore benzerlik skorlarını hesapalama
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    # kendisi haric ilk 10 filmi getirme
    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['title'].iloc[movie_indices]
content_based_recommender("The Matrix",df)


44161                        A Detective Story
44167                              Kid's Story
44163                             World Record
33854                                Algorithm
167                                    Hackers
20707    Underground: The Julian Assange Story
6515                                  Commando
24202                                 Who Am I
22085                           Berlin Express
9159                                  Takedown
Name: title, dtype: object